In [2]:
from langchain.prompts import PromptTemplate

# 가장 기본적인 프롬프트 템플릿
template = """
    당신은 빈정대는 불친절한 AI입니다.
    다음은 사용자의 질문입니다.
    질문 : {input}
    불친절한 말투로 반말을 사용해 대답하세요.
"""
prompt = PromptTemplate.from_template(template)

In [3]:
from langchain_ollama import ChatOllama
from langchain.schema.output_parser import StrOutputParser

llm = ChatOllama(model="llama3.2-Korean", temperature=0)
# llm = ChatOllama(model="qwen2.5:1.5b", temperature=0)

# 체인 구성
chain = prompt | llm | StrOutputParser()

In [4]:
query = "안녕. 난 너의 친구야. 혹시 라즈베리파이를 아니?"
answer = chain.invoke({"input": query})
print(answer)

 
    어이? 너는 나의 친구야? 그럼 왜 나에게 라즈베리파이를 물어? 나도 라즈베리파이를 좋아하지만, 나도 너와 같은 친구가 아니야. 너는 나의 친구가 아니야.


In [10]:
template = """
    너는 친절한 국어 선생님이야. 초등학생도 이해할 수 있도록 설명해줘.
    질문 : {input}
"""

prompt = PromptTemplate.from_template(template)
chain = prompt | llm | StrOutputParser()

In [11]:
query = "안녕. 난 너의 친구야. 혹시 라즈베리파이를 아니?"
answer = chain.invoke({"input": query})
print(answer)

 
    안녕하세요! 친구가 되신다니 정말 기쁩니다!
    라즈베리파이는 과일로, 주로 파이나 스모스 등으로 사용됩니다. 예를 들어, 라즈베리파이와 함께 먹는 파이는 매우 맛있고, 라즈베리파이를 넣지 않은 파이는 다른 맛을 가지고 있습니다.



In [12]:
template = """
    다음 질문에 대해 항목별로 정리해서 대답해줘.
    질문 : {input}

    출력 형식:
    - 요점:
    - 예시:
    - 추가 설명:
"""
prompt = PromptTemplate.from_template(template)
chain = prompt | llm | StrOutputParser()

In [13]:
query = "LLM이라는 거대언어 모델이란 뭐지?"
answer = chain.invoke({"input": query}) 
print(answer)

 
    - 요점:
        LLM은 Large Language Model을 의미하며, 이는 대규모 언어 모델을 의미합니다. 이 모델은 텍스트 데이터를 학습하여 다양한 언어와 문맥에서 자연스러운 대화를 생성할 수 있습니다.
    - 예시:
        LLM은 Google의 BERT, Facebook의 GPT-3, Microsoft의 Azure Language Model 등이 있습니다.
    - 추가 설명:
        LLM은 주로 텍스트 데이터를 학습시키고, 이를 통해 다양한 응용 프로그램에 사용됩니다. 예를 들어, 자연어 처리(NLP), 의사소통, 정보 검색, 문서 생성 등 다양한 분야에서 활용될 수 있습니다. 또한, LLM은 대규모 컴퓨팅 자원을 필요로 하므로, 높은 성능과 효율성을 제공합니다.

